In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
orders = pd.read_csv('data/orders.csv')
customers = pd.read_csv('data/customers.csv')
products = pd.read_csv('data/products.csv')
categories = pd.read_csv('data/categories.csv')
order_items = pd.read_csv('data/order_items.csv')
reviews = pd.read_csv('data/reviews.csv')
inventory_logs = pd.read_csv('data/inventory_logs.csv')

In [ ]:
print("\norders")
orders.info()
print("\ncustomers")
customers.info()
print("\ncategories")
categories.info()
print("\nproducts")
products.info()
print("\norder_items")
order_items.info()
print("\ninventory_logs")
inventory_logs.info()
print("\nreviews")
reviews.info()

In [ ]:
def normalize_string_column_values(df, column_name):
    df[column_name] = df[column_name].str.upper()
    df[column_name] = df[column_name].str.strip()
    df[column_name] = df[column_name].str.replace(' ', '_')
    return df

### 1. ¿Cuál es el Estado que más descuentos tiene en total? ¿y en promedio?

In [ ]:
orders["billing_address"] = orders["billing_address"].str.upper()
orders.fillna({"billing_address":"UNDEFINED"}, inplace=True)

# Los valores de "billing_address" se caracterizan por tener en alguna parte
# 2 letras mayúsculas seguidas de un espacio y 5 dígitos.
# Interpreto las dos letras mayúsculas como el Estado y los 5 dígitos como el código postal.
pattern = r'([A-Z]{2})\s(\d{5})'
orders[["state", "zip_code"]] = orders["billing_address"].str.extract(pattern)

In [ ]:
# Chequeo que la extracción haya salido bien
print("Filas totales en dataset orders:", len(orders))
print("Filas con estado no nulos:", orders["state"].notna().sum())
print("Filas con estado nulo:", orders["state"].isna().sum())

null_state_and_null_billing_addr = orders["state"].isna() & orders["billing_address"].str.contains("UNDEFINED")

print("¿Todas las filas que tienen estado nulo, tienen dirección de facturación indefinida?", 
        "Si" if null_state_and_null_billing_addr.sum() == orders["state"].isna().sum() else "No")

In [ ]:
# Suposición: Si el valor de la columna "discount_amount" es nulo, es porque no hay descuento.
orders.fillna({"discount_amount":0.0}, inplace=True)

In [ ]:
import json
with open("state_names.json", "r") as f:
    state_names = json.load(f)

orders["state_name"] = orders["state"].map(state_names)

In [ ]:
# Filtrar estados militares y otros
# filtro estos valores por interpretación de enunciado, 
# supuse que se refería a los 50 estados de EEUU
not_states_filter = ~(
    orders["state"].str.contains("AA")   # Military Americas
    | orders["state"].str.contains("AE") # Military Europe
    | orders["state"].str.contains("AP") # Military Pacific
    | orders["state"].str.contains("FM") # Federated States of Micronesia
    | orders["state"].str.contains("MH") # Marshall Islands
    | orders["state"].str.contains("MP") # Northern Mariana Islands
    | orders["state"].str.contains("PW") # Palau
    | orders["state"].str.contains("GU") # Guam
    | orders["state"].str.contains("VI") # U.S. Virgin Islands
    | orders["state"].str.contains("AS") # American Samoa
    | orders["state"].isna()             # Nulos
)
orders_with_discount = orders[orders["discount_amount"] > 0].loc[not_states_filter]

In [ ]:
quantity_of_orders_with_discounts_by_state = orders_with_discount.groupby("state")["order_id"].count().reset_index()
quantity_of_orders_with_discounts_by_state.rename(columns={"order_id": "no_of_discounts"}, inplace=True)
quantity_of_orders_with_discounts_by_state["state_name"] = quantity_of_orders_with_discounts_by_state["state"].map(state_names)

print("\nTop 5 estados con más órdenes con descuentos:")
quantity_of_orders_with_discounts_by_state.nlargest(5, "no_of_discounts")

In [ ]:
states_avg_discount = orders_with_discount.groupby("state")["discount_amount"].mean().reset_index()
states_avg_discount.rename(columns={"discount_amount": "avg_discount"}, inplace=True)
states_avg_discount["state_name"] = states_avg_discount["state"].map(state_names)

avg_discount = orders_with_discount["discount_amount"].mean()

print("\nDescuento promedio de las órdenes en todos los estados:", avg_discount)

print("\nTop 5 estados con mayor descuento promedio en las órdenes:")
states_avg_discount.nlargest(5, "avg_discount")

### 2. ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado 'Refunded'? 
###    ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

In [ ]:
orders.fillna({"status":"UNDEFINED"}, inplace=True)
orders = normalize_string_column_values(orders, "status")
orders["status"].value_counts()

In [ ]:
refunded_orders = orders[orders["status"].str.contains("REFUNDED")]

amount_refunded_orders_by_zipcode = refunded_orders["zip_code"].value_counts().reset_index()
top_refunded_zipcodes = amount_refunded_orders_by_zipcode.nlargest(5, "count")

print("\nTop 5 códigos postales más comunes para órdenes con estado 'Refunded':")
print(top_refunded_zipcodes)

print("\nCódigos postales con 5 órdenes 'Refunded':")
print(amount_refunded_orders_by_zipcode.loc[amount_refunded_orders_by_zipcode["count"] == 5])


In [ ]:
# Acá hay dos formas de hacer "lo mismo", pruebo utilizando merge y usando isin
refunded_clients_ids = top_refunded_zipcodes.merge(orders, left_on="zip_code", right_on="zip_code")["customer_id"].unique()
refunded_clients_names = customers[customers["customer_id"].isin(refunded_clients_ids)][["customer_id", "first_name"]]

most_common_names = refunded_clients_names["first_name"].value_counts().reset_index().nlargest(5, "count")

print("\nTop 5 nombres más comunes entre los clientes que realizaron órdenes 'Refunded' en los códigos postales con más órdenes 'Refunded':")
print(most_common_names)

## 3. Para cada tipo de pago y segmento de cliente, 
## devolver la suma y el promedio expresado como porcentaje 
## de clientes activos y de consentimiento de marketing

In [ ]:
# Limpio los datos y homogeinizo los datos
customers = normalize_string_column_values(customers, "customer_segment")
orders = normalize_string_column_values(orders, "payment_method")

In [ ]:
# Mergeo los dos dataframes porque necesito columnas de ambos:
# de customers: is_active, marketing_consent, customer_segment
# de orders:    payment_method
orders_customers = orders.merge(customers, on="customer_id").reset_index()

# Paso las dos columnas booleanas a enteros para poder calcular el porcentaje 
# a través del promedio
orders_customers["is_active"] = orders_customers["is_active"].astype(int)
orders_customers["marketing_consent"] = orders_customers["marketing_consent"].astype(int)

# Me quedo con filas únicas por combinación de método de pago, segmento de cliente y customer_id
# para no contar dos veces a un mismo cliente que hizo varias órdenes con el mismo método de pago
orders_customers_unique = orders_customers.drop_duplicates(subset=["payment_method", "customer_segment", "customer_id"])

orders_customers_grouped = orders_customers_unique.groupby(["payment_method", "customer_segment"]).agg({
    "customer_id": "count",
    "is_active": ["sum", "mean"],
    "marketing_consent": ["sum", "mean"]
})
orders_customers_grouped[('is_active', 'mean')] = orders_customers_grouped[('is_active', 'mean')] * 100
orders_customers_grouped[('marketing_consent', 'mean')] = orders_customers_grouped[('marketing_consent', 'mean')] * 100
orders_customers_grouped

In [ ]:
orders_customers_grouped.columns = [
    'Total de usuarios',
    'Total de usuarios activos',
    'Porcentaje de usuarios activos',
    'Total de consentimientos de marketing',
    'Porcentaje de consentimientos de marketing'
]
orders_customers_grouped.index.set_names(['Método de Pago', 'Segmento de Cliente'], inplace=True)
orders_customers_grouped

In [ ]:
client_by_payment_segment = orders_customers_grouped['Total de usuarios activos'].unstack()
plt.figure(figsize=(12, 6))
plt.title('Cantidad de usuarios activos por método de pago y segmento de cliente')
sns.heatmap(client_by_payment_segment, annot=True, fmt=".0f", cmap="YlGnBu", linecolor="white", linewidths=0.6)

### 4. Para los productos que contienen en su descripción la palabra "stuff",
### calcular el peso total de su inventario agrupado por marca,
### mostrar sólo la marca y el peso total de las 5 más pesadas

In [ ]:
products["description"] = products["description"].fillna("")
stuff_products = products.loc[products["description"].str.contains("STUFF", case=False)][["product_id", "brand", "weight_kg"]]

### Reason y Movement type

In [ ]:
inventory_logs = normalize_string_column_values(inventory_logs, "reason")
inventory_logs = normalize_string_column_values(inventory_logs, "movement_type")

In [ ]:
inventory_logs.groupby(["movement_type", "reason"]).size().unstack()

In [ ]:
categories = normalize_string_column_values(categories, "category_name")
categories = normalize_string_column_values(categories, "parent_category")
category_names = categories[["category_id", "category_name", "parent_category"]]
category_names = category_names.fillna("UNDEFINED")
products = products.merge(category_names, on="category_id", how="left")

In [ ]:
products.groupby(["parent_category", "category_name"]).size()

In [ ]:
orders["order_date"] = pd.to_datetime(orders["order_date"], format="%Y-%m-%dT%H:%M:%S.%f", errors="coerce").dt.date
orders["year"] = pd.DatetimeIndex(orders["order_date"]).year
orders["month"] = pd.DatetimeIndex(orders["order_date"]).month
orders["year_month"] = pd.to_datetime(orders[["year", "month"]].assign(day=1)).dt.strftime('%Y-%m')

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Cantidad de órdenes por mes y año")
sns.lineplot(
    data=orders["order_date"].value_counts().reset_index(), 
    x="order_date", y="count", color="blue", label="Total")
sns.lineplot(
    data=orders.loc[orders["discount_amount"] > 0]["order_date"].value_counts().reset_index(), 
    x="order_date", y="count", color="green", label="Con descuento")
sns.lineplot(
    data=orders.loc[orders["discount_amount"] == 0]["order_date"].value_counts().reset_index(), 
    x="order_date", y="count", color="orange", label="Sin descuento")
plt.xlabel("Fecha de la orden")
plt.ylabel("Cantidad de órdenes")

In [ ]:
orders_per_state_year_month = orders.groupby(["state","year_month"]).size().reset_index().rename(columns={0: "count"})
plt.figure(figsize=(15, 8))
sns.barplot(data=orders_per_state_year_month, x="year_month", y="count", hue="state")